In [ ]:
## from Gugger, S., & Howard, J. (2020). Deep Learning for Coders with Fastai and PyTorch: AI Applications Without a PhD. O’Reilly Media.
from fastbook import *
from fastai.vision.widgets import *
import icrawler
import lxml
from icrawler.builtin import GoogleImageCrawler

In [ ]:
#download data
categories = "cat", "Maine Coon", "random thing"

path = Path("images/cats")

#if not path.exists():
#   path.mkdir()
for o in categories:
    dest = (path/o)
    dest.mkdir(exist_ok=True)
    google_crawler = GoogleImageCrawler(parser_threads=2, downloader_threads=4, storage={'root_dir': dest})
    google_crawler.crawl(keyword=o, max_num=250)

In [ ]:
#refactor to prep for DataLoaders and 
#randomly crop photos between epochs so
#"our model can learn to focus on, and recognize, different features in our images."
## - https://github.com/fastai/fastbook/blob/master/02_production.ipynb
cats = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(224))
cats = cats.new(item_tfms=RandomResizedCrop(224, min_scale=0.3), batch_tfms=aug_transforms())

#load and show some data from DataBlock object
dls = cats.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=1, unique=True)

In [ ]:
## The desired output of this model is to determine whether or not an image is of a cat who we call Katja.
### some issues, misunderstandings and concerns i have with this model are:
#### Initially i wanted to train the model on images of Katja
##### against a single, nonKatja category, whose set might've consisted of a number
##### of different cat photos and of photos which weren't cats.
##### Not sure how this would've compared to the current setup of four categories.
#### I suppose I also wanted this model to determine if an image was of a nonCat in addition to nonKatja.
##### Not sure how effective it is at this.
#### This is model is inherently biased toward labeling Katja as a Maine Coon.
#### My apologies for being non-scientific. Working with data like this is very new to me.

In [ ]:
#train the model
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
### some interpretation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
#get pickle (.pkl) file
learn.export()

In [ ]:
### some prediction

In [ ]:
path = Path()
print(path)
path.ls(file_exts='.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
import requests
img_data = requests.get("https://wallup.net/wp-content/uploads/2016/01/180325-animals-fox.jpg").content
with open('images/image_name.jpg', 'wb') as handler:
    handler.write(img_data)


In [ ]:
learn_inf.predict('images/image_name.jpg')

In [ ]:
learn_inf.predict('images/predictKatja/IMG_0159.JPEG')

In [ ]:
learn_inf.predict('images/random_thing.jpg')